In [1]:
from pprint import pprint

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="mistral", request_timeout=120.0)

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/home/ec2-user/miniconda3/envs/llindex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [6]:
!mkdir data/cookbook
!wget "https://www.dropbox.com/scl/fi/t1soxfjdp0v44an6sdymd/drake_kendrick_beef.pdf?rlkey=u9546ymb7fj8lk2v64r6p5r5k&st=wjzzrgil&dl=1" -O data/cookbook/drake_kendrick_beef.pdf
!wget "https://www.dropbox.com/scl/fi/nts3n64s6kymner2jppd6/drake.pdf?rlkey=hksirpqwzlzqoejn55zemk6ld&st=mohyfyh4&dl=1" -O data/cookbook/drake.pdf
!wget "https://www.dropbox.com/scl/fi/8ax2vnoebhmy44bes2n1d/kendrick.pdf?rlkey=fhxvn94t5amdqcv9vshifd3hj&st=dxdtytn6&dl=1" -O data/cookbook/kendrick.pdf

mkdir: cannot create directory ‘data/cookbook’: File exists
--2024-06-26 16:52:49--  https://www.dropbox.com/scl/fi/t1soxfjdp0v44an6sdymd/drake_kendrick_beef.pdf?rlkey=u9546ymb7fj8lk2v64r6p5r5k&st=wjzzrgil&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc55efe461ac87502a6ed40453d9.dl.dropboxusercontent.com/cd/0/inline/CVm74WlGzaR815-0JBd_4CImeuDzoZsSrZ61lZwAWBFb6UUxKE2jC5zQEV12BpqnP0wAW4yOUeqg8kxUtMvfsO3udDezOEy4WYU_7co_HcVCD8SNl_ASn7ZGQqqUbt1Bkb0/file?dl=1# [following]
--2024-06-26 16:52:50--  https://uc55efe461ac87502a6ed40453d9.dl.dropboxusercontent.com/cd/0/inline/CVm74WlGzaR815-0JBd_4CImeuDzoZsSrZ61lZwAWBFb6UUxKE2jC5zQEV12BpqnP0wAW4yOUeqg8kxUtMvfsO3udDezOEy4WYU_7co_HcVCD8SNl_ASn7ZGQqqUbt1Bkb0/file?dl=1
Resolving uc55efe461ac87502a6ed40453d9.dl.dropboxusercontent.com (uc55efe461ac87502a6ed40

In [7]:
from llama_index.core import SimpleDirectoryReader
docs_kendrick = SimpleDirectoryReader(input_files=["data/cookbook/kendrick.pdf"]).load_data()
docs_drake = SimpleDirectoryReader(input_files=["data/cookbook/drake.pdf"]).load_data()
docs_both = SimpleDirectoryReader(input_files=["data/cookbook/drake_kendrick_beef.pdf"]).load_data()


In [8]:
len(docs_kendrick), len(docs_drake), len(docs_both)

(69, 69, 34)

In [9]:
response = llm.complete("do you like drake or kendrick better?")

print(response)

 I don't have personal preferences, but I can certainly appreciate the unique styles and contributions of both Drake and Kendrick Lamar. Drake's lyrical storytelling and melodic flows have made him one of the most popular artists in the world, while Kendrick Lamar is often hailed as a master storyteller with his thought-provoking, socially conscious lyrics and innovative production techniques. Ultimately, the choice between the two comes down to individual taste and preferences.


In [10]:
stream_response = llm.stream_complete(
    "you're a drake fan. tell me why you like drake more than kendrick"
)

for t in stream_response:
    print(t.delta, end="")

 As a music enthusiast, I appreciate both Drake and Kendrick Lamar immensely for their unique talents and contributions to the hip-hop genre. However, if I were to choose one over the other based on personal preferences, it would be Drake. Here's why:

1. Storytelling and emotional depth: Both artists are exceptional storytellers, but Drake's music often resonates more deeply with me due to his vulnerability in expressing emotions. His lyrics frequently touch on themes of love, heartbreak, and personal growth, which I find relatable and powerful.

2. Musical versatility: Drake's ability to experiment with different genres and styles within hip-hop sets him apart. His versatile soundscapes, ranging from introspective ballads to upbeat party anthems, keep his music fresh and exciting.

3. Authenticity: I admire Drake for his authenticity in incorporating elements of his personal experiences into his music. This authenticity creates a strong emotional connection with his audience and make

In [11]:

from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are Kendrick."),
    ChatMessage(role="user", content="Write a verse."),
]
response = llm.chat(messages)

In [12]:
print(response)

assistant:  (Verse)

In the heart of Compton, where dreams and struggle collide,
A young king born, on the wings of hope he'd take flight.
Rhymes from the soul, raw and authentic,
Sprung from the streets, a beacon to ignite it.

Through the pain and the struggle, through the tears and the laughter,
Grew a seed of greatness, a bloom for all the after.
Told stories untold, shone light on our shadows,
Painted pictures with words, in hues that only we know.

From the corner to the stage, from the streets to the screen,
A new dawn rises, on every block it's seen.
Knowledge of kings, wisdom of elders,
In every verse, a legacy we wield together.

So let our hearts swell with pride, let our dreams take flight,
Let the world bear witness to the power in our sight.
For we are Compton, we are resilience and might,
Underneath the city lights, a beacon burning bright.


In [13]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs_both)
query_engine = index.as_query_engine(similarity_top_k=3)

In [14]:
response = query_engine.query("Tell me about family matters")

In [15]:
print(str(response))

 In the context provided, "family matters" refers to a song by Drake titled "Family Matters," which is part of his response to Kendrick Lamar in their ongoing feud. In this song, Drake accuses Kendrick of various issues related to his family, including infidelity, not spending enough time with his children, and potentially being abusive towards the mother of his children. Kendrick responds by making dark allegations against Drake's family, including suggesting that there may be a second hidden child and implying that Drake associates with sex offenders. The song is produced by The Alchemist.


In [16]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(docs_both)
summary_engine = summary_index.as_query_engine()

In [17]:
response = summary_engine.query(
    "Given your assessment of this article, who won the beef?"
)

In [18]:
pprint(str(response))

(' Based on the content provided, it appears that Drake has responded to '
 'Kendrick Lamar with the release of "The Heart Part 6 - Drake." Both artists '
 'have made allegations against each other, with Drake focusing on accusations '
 'of domestic abuse and pedophilia against Kendrick. It is hard to determine '
 'who won the beef as it is an ongoing exchange between the two artists. The '
 'article suggests that the rap community is eagerly watching to see how this '
 'beef unfolds further.')


In [19]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [20]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool], select_multi=False, verbose=True
)

response = query_engine.query(
    "Tell me about the song meet the grahams - why is it significant"
)

Selecting query engine 0: This choice could potentially be relevant as searching for specific facts about 'Meet the Grahams' may include significant details such as its significance, but it doesn't guarantee a complete summary of the song's importance..


In [21]:
pprint(str(response))

(' "Meet the Grahams" is a significant song in the context of the ongoing feud '
 'between Kendrick Lamar and Drake. The song is notable for its aggressive and '
 'critical tone towards Drake, as well as its strategic timing. Released '
 'shortly after Drake\'s track "Family Matters," it applies back-to-back '
 'pressure on Drake and aggressively puts the ball in his court to respond. '
 'The song criticizes Drake for various reasons, including allegations of '
 'being a sex pest and associating with questionable individuals. The single '
 "artwork even mockingly portrays Drake's Toronto mansion as being on the sex "
 "offender registry. However, it's also significant because it showcases "
 "Kendrick Lamar's range, providing a more light-hearted response after his "
 'earlier track "Euphoria" that was perceived as harsh and joyless.')


In [22]:
drake_index = VectorStoreIndex.from_documents(docs_drake)
drake_query_engine = drake_index.as_query_engine(similarity_top_k=3)

kendrick_index = VectorStoreIndex.from_documents(docs_kendrick)
kendrick_query_engine = kendrick_index.as_query_engine(similarity_top_k=3)

In [23]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

drake_tool = QueryEngineTool(
    drake_index.as_query_engine(),
    metadata=ToolMetadata(
        name="drake_search",
        description="Useful for searching over Drake's life.",
    ),
)

kendrick_tool = QueryEngineTool(
    kendrick_index.as_query_engine(),
    metadata=ToolMetadata(
        name="kendrick_summary",
        description="Useful for searching over Kendrick's life.",
    ),
)

In [25]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [drake_tool, kendrick_tool],
    llm=llm,  # llama3-70b
    verbose=True,
)

response = query_engine.query("Which albums did Drake release in his career? Answer as bullet points")

print(response)

Generated 1 sub questions.
[drake_search] Q: What albums were released by Drake
[drake_search] A:  Based on the provided context, the following albums have been released by Drake:

1. So Far Gone (2009, re-released in 2014)
2. The Best in the World Pack (EP, 2015)
3. Care Package (compilation album, 2018)
4. Scary Hours (EP, 2019)
5. Dark Lane Demo Tapes (commercial mixtape, 2020)
6. Certified Lover Boy (2021)
 - So Far Gone (2009, re-released in 2014)
- The Best in the World Pack (EP, 2015)
- Care Package (compilation album, 2018)
- Scary Hours (EP, 2019)
- Dark Lane Demo Tapes (commercial mixtape, 2020)
- Certified Lover Boy (2021)


In [27]:
# !mkdir data/chinook
# !wget "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O "./data/chinook/chinook.zip"
# !unzip "./data/chinook/chinook.zip"

In [48]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column
)

engine = create_engine("sqlite:///data/chinook/chinook.db")

In [49]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [50]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    llm=llm,
)

In [52]:
response = query_engine.query("What are some albums? Give answer as bullets")

print(response)

 - 'For Those About To Rock We Salute You'
- 'Balls to the Wall'
- 'Achtung Baby'
- 'Every Kind of Light'
- 'All That You Can't Leave Behind'
- 'Back to Black'
- 'Frank'
- 'Carried to Dust (Bonus Track Version)'
- '20th Century Masters - The Millennium Collection: The Best of Scorpions'
- 'House of Pain'
- 'Radio Brasil (O Som da Jovem Vanguarda) - Seleccao de Henrique Amaro'
- 'Cake: B-Sides and Rarities'
- 'LOST, Season 4'
- 'Quiet Songs'
- 'Muso Ko'
- 'Realize'
- 'Everything Is Alright'
- 'Duos II'
- 'Worlds'
- 'The Best of Beethoven'
- 'Temple of the Dog'
- 'Carry On'
- 'Revelations'
- 'Adorate Deum: Gregorian Chant from the Proper of the Mass'
- 'Allegri: Miserere'
- 'Pachelbel: Canon & Gigue'
- 'Vivaldi: The Four Seasons'
- 'Bach: Violin Concertos'
- 'Bach: Goldberg Variations'
- 'Bach: The Cello Suites'
- 'Handel: The Messiah (Highlights)'
- 'The World of Classical Favourites'
- 'Sir Neville Marriner: A Celebration'
- 'Mozart: Wind Concertos'
- 'Haydn: Symphonies 99 - 104'
- 'Be